# Langchain Expression Language (LCEL)

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Simple Chain

In [3]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [5]:
prompt = PromptTemplate.from_template(
    "tell me a joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [6]:
chain = prompt | model | output_parser

In [9]:
chain.invoke({"topic": "machine learning"})

'Why did the machine learning algorithm go to therapy?\n\nBecause it had too many unresolved issues with its training data!'

## More Complex Chain

In [26]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnableMap
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [22]:
model = ChatOpenAI()

In [12]:
db = DocArrayInMemorySearch.from_texts(
    texts=[
        "Playstation 5 has a 1.8 ghz CPU, 16 GB unified memory and a 825 GB SSD",
        "kittens are cute and furry"
    ],
    embedding=OpenAIEmbeddings()
)

In [36]:
retriever = db.as_retriever(search_kwargs={"k": 1})
retriever.get_relevant_documents("what do you know about cats?")

[Document(page_content='kittens are cute and furry')]

In [37]:
prompt = PromptTemplate.from_template("""
Answer the following question only using the below context:

{context}

Question: {question}
"""
)

In [38]:
chain = RunnableMap(
    {
        "question": lambda x: x["question"],
        "context": lambda x: retriever.get_relevant_documents(x["question"])
    }
) | prompt | model | StrOutputParser()

In [39]:
chain.invoke({"question": "How much disk space does playstation 5 have?"})

'Playstation 5 has 825 GB of disk space.'

In [40]:
chain.invoke({"question": "What do you know about cats?"})

'Based on the given context, I know that kittens are cute and furry.'

In [41]:
inputs = RunnableMap(
    {
        "question": lambda x: x["question"],
        "context": lambda x: retriever.get_relevant_documents(x["question"])
    }
) 

In [43]:
result = inputs.invoke({"question": "What do you know about cats?"})
result

{'question': 'What do you know about cats?',
 'context': [Document(page_content='kittens are cute and furry')]}

In [49]:
print(prompt.invoke(input=result))

text="\nAnswer the following question only using the below context:\n\n[Document(page_content='kittens are cute and furry')]\n\nQuestion: What do you know about cats?\n"


## Bind

In [56]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

In [53]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [60]:
prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}")
])

In [61]:
model = ChatOpenAI()
model = model.bind(functions=functions)

In [62]:
chain = prompt | model

In [63]:
chain.invoke({"input": "What is the weather in ghaziabad?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': '{\n  "airport_code": "DEL"\n}'}})

In [64]:
chain.invoke({"input": "How much runs did India score in the world cup final?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'sports_search', 'arguments': '{\n  "team_name": "India"\n}'}})

## Fallbacks

In [67]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import json

In [66]:
simple_model = OpenAI(
    model="text-davinci-001",
    temperature=0,
    max_tokens=1000
)

In [69]:
prompt = """write three poems in a json blob, where each poem \
is a json blob of a title, author, and first line"""

In [70]:
simple_chain = simple_model | json.loads

In [71]:
simple_chain.invoke(prompt)

JSONDecodeError: Extra data: line 5 column 1 (char 103)

In [72]:
newer_model = ChatOpenAI(temperature=0)
fallback_chain = newer_model | StrOutputParser() | json.loads

In [73]:
fallback_chain.invoke(prompt)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

In [74]:
chain_with_fallback = simple_chain.with_fallbacks([fallback_chain])

In [75]:
chain_with_fallback.invoke(prompt)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

## Interface

In [82]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a joke about {topic}"
)
model = ChatOpenAI()
chain = prompt | model | StrOutputParser()

In [83]:
chain.invoke({"topic": "cats"})

"Why don't cats play poker in the wild?\n\nToo many cheetahs!"

In [84]:
chain.batch([{"topic": "cats"}, {"topic": "video games"}])

["Why don't cats play poker in the wild?\n\nToo many cheetahs!",
 'Why did the gamer go broke?\n\nBecause he couldn\'t find the right way to "coin" the phrase!']

In [86]:
for t in chain.stream({"topic": "cats"}):
    print(t)

In [87]:
response = await chain.ainvoke({"topic": "cats"})
response

'Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!'